In [1]:
import numpy as np

from scipy.spatial.transform import Rotation as R

from reachy_pyluos_hal.orbita_kinematic_model import OrbitaKinematicModel

In [2]:
def inverse(roll, pitch, yaw):
    a = OrbitaKinematicModel(R0=np.eye(3))
    q = R.from_euler('xyz', [roll, pitch, yaw]).as_quat()
    return np.deg2rad(a.get_angles_from_quaternion(q[3], q[0], q[1], q[2]))

## Generate dataset

In [ ]:
N = int(180 * 0.5 + 1)
print(N ** 3)

X, Y = [], []

space = np.deg2rad(np.linspace(-60, 60, N))
yaw_space = np.deg2rad(np.linspace(-180, 180, 2 * N))


for roll in space:
    for pitch in space:
        for yaw in space:
            try:
                d = inverse(roll, pitch, yaw)
                X.append(d)
                Y.append((roll, pitch, yaw))
            except ValueError:
                pass

X = np.array(X)
Y = np.array(Y)

X.shape, Y.shape

In [ ]:
np.rad2deg(X), np.rad2deg(Y)

Save the data if needed.

In [ ]:
# np.save(f'data/X-{N}.npy', X)
# np.save(f'data/Y-{N}.npy', Y)

Different strategies are available to generate a model predicting the forward kinematics.

## Using MLP Regressor from sklearn

### (model currently used in reachy_pyluos_hal (orbita_kinematic_model.py))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score

Load the data it it was previously generated.

In [ ]:
X, Y = np.load('X-91.npy'), np.load('Y-91.npy')

X.shape, Y.shape
Xtrig=np.array([np.sin(X),np.cos(X)]).reshape((X.shape[0],X.shape[1]*2))
Ytrig=np.array([np.sin(Y),np.cos(Y)]).reshape((Y.shape[0],Y.shape[1]*2))

In [ ]:
scaler = StandardScaler()

In [ ]:
print(X.shape)
print(Y.shape)
scaler.fit(X,Y)

In [ ]:
scaler.transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model = MLPRegressor(hidden_layer_sizes=(300,))
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_test, y_test))  # returns 0.99964

In [ ]:
import time
test=np.array([0,0,0]).reshape((1,3))
test=np.radians(test)
t0=time.time()
res=model.predict(test)
t1=time.time()
print(f'RES: {res} {t1-t0}')  # returns RES: [[-0.00390054  0.00119692  0.00281156]] 0.0001933574676513672

Save the trained model.

In [ ]:
import pickle

filehandler = open("mlpreg.obj","wb")
pickle.dump(model,filehandler)

## Other methods

### Training a keras model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
D = np.hstack((X, Y))
np.random.shuffle(D)

I = int(len(D) * 0.8)
X_train, Y_train = D[:I, :3], D[:I, 3:]
X_val, Y_val = D[I:, :3], D[I:, 3:]

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

In [ ]:
inputs = keras.Input(shape=(3,))
x = layers.Dense(units=16, activation='relu')(inputs)
outputs =layers.Dense(units=3, activation='linear')(x)

m = keras.Model(inputs, outputs)
m.compile(optimizer='Adam', loss='mse')

In [ ]:
m.summary()

In [ ]:
m.fit(X_train, Y_train, epochs=10)

In [ ]:
m.evaluate(X_val, Y_val)

### Using KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X, Y)

Measure the error.

In [ ]:
e = []

for _ in range(1000):
    try:
        roll, pitch, yaw = np.deg2rad(np.random.rand(3) * 120 - 60)
        y = np.array([[roll, pitch, yaw]])

        disks = inverse(roll, pitch, yaw)
        x = np.array([disks])

        y_hat = knn.predict(x)

        e.append(np.linalg.norm(y - y_hat))
    except ValueError:
        pass

e = np.array(e)
print(np.rad2deg(e).max()) # returns 1.12 deg